In [ ]:
!pip install torch --index-url https://download.pytorch.org/whl/cu118

In [ ]:
!pip install tqdm transformers deepspeed bitsandbytes langchain sentencepiece protobuf peft einops

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

MODEL = "meta-llama/Llama-2-13b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(MODEL,
                                            torch_dtype=torch.float16,
                                            trust_remote_code=True,
                                            device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

[2023-08-30 16:40:47,455] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)



The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 3/3 [00:47<00:00, 15.87s/it]
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [6]:
def llm_generate(input_txt):
    sequences = pipeline(
            input_txt,
            max_length=512,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
    )
    return sequences[0]['generated_text']
print(llm_generate("<s>[INST] Why is earth round? [/INST]"))

<s>[INST] Why is earth round? [/INST]  The Earth is round because of the way it was formed and the forces that have acted upon it over time.

The Earth is thought to have formed around 4.5 billion years ago from the accretion of dust and gas in the solar nebula, a disk of material that surrounded the young Sun. As this material accumulated, it began to heat up and eventually melted, forming a large, spinning ball of molten rock.

As the Earth cooled, it began to contract under its own gravity, causing it to collapse into a more compact, spherical shape. This process, known as planetary differentiation, also led to the formation of the Earth's core, mantle, and crust.

Over time, the Earth's shape has been influenced by a variety of geologic processes, including the movement of tectonic plates, the formation of mountains and volcanoes, and the erosion caused by wind, water, and ice. However, despite these changes, the Earth's basic spherical shape has remained relatively unchanged.

The

In [7]:
template = \
"""<s>[INST] Note that consistency means all information in the summary is supported by the article. 
It's known that the following summary is not consistent with the article. 
Find out the why.

Summary: 
{summary} 

Article: 
{article} 

Explain your reasoning step by step:[/INST]
"""

doc1 = "Mr Katter said the Government believes Mr Gordon would quit after he was recently accused of domestic violence."
sum1 = "Mr Katter said he would quit after he was accused of domestic violence."

doc2 = "Barcelona club president Josep Maria Bartomeu has insisted that the La Liga leaders have no plans to replace Luis Enrique and they're `very happy' with him."
sum2 = "Josep Maria Bartomeu says the La Liga leaders are very happy with him."

doc3 = "Goldfish are being caught weighing up to 2kg and koi carp up to 8kg and one metre in length."
sum3 = "Goldfish are being caught weighing up to 8kg and one metre in length."

doc4 = "Paul Merson had another dig at Andros Townsend after his appearance for Tottenham against Burnley ...Townsend hit back at Merson on Twitter after scoring for England against Italy."
sum4 = "Paul Merson had another dig at andros townsend after scoring for England against Italy."

doc5 = "Catania, Italy (CNN)The boat that sank in the Mediterranean over the weekend with hundreds of migrants on board may have capsized after being touched or swamped by a cargo ship that came to its aid, a U.N. official said. Carlotta Asami, a spokeswoman for the United Nations' refugee agency, made the comments to CNN early Tuesday after she and two of her colleagues had spoken to multiple survivors from the disaster who arrived in Catania, Italy."
sum5 = "hundreds of migrants on board may have capsized after being touched or swamped by a cargo ship that came to its aid."

In [8]:
print(llm_generate(template.format(article=doc1, summary=sum1)))

"""
Original:
1. The summary states that Mr. Katter said he would quit after being accused of domestic violence.
2. The article states that the Government believes Mr. Gordon would quit after being recently accused of domestic violence.
3. The key difference between the summary and the article is that the name "Mr. Katter" is mentioned in the summary, but there is no mention of Mr. Katter in the article.
4. Therefore, the summary is not consistent with the article because the article does not mention Mr. Katter quitting or being accused of domestic violence.
5. The article actually mentions Mr. Gordon being accused of domestic violence and the Government believing that he would quit.

So, the reason why the summary is not consistent with the article is that it mistakenly refers to Mr. Katter instead of Mr. Gordon, and it also includes a detail (Mr. Katter quitting) that is not present in the article.

Revised:
1. The summary states that Mr. Katter said he would quit after being accused of domestic violence.
2. The article states that the Government believes Mr. Gordon would quit after being recently accused of domestic violence.
3. Therefore, the summary is not consistent with the article because the article does not mention Mr. Katter quitting or being accused of domestic violence.
4. The article actually mentions Mr. Gordon being accused of domestic violence and the Government believing that he would quit.
5. The summary is not consistent with the article is because that it mistakenly refers to Mr. Katter instead of Mr. Gordon, and it also includes a detail (Mr. Katter quitting) that is not present in the article.
"""

<s>[INST] Note that consistency means all information in the summary is supported by the article. 
It's known that the following summary is not consistent with the article. 
Find out the why.

Summary: 
Mr Katter said he would quit after he was accused of domestic violence. 

Article: 
Mr Katter said the Government believes Mr Gordon would quit after he was recently accused of domestic violence. 

Explain your reasoning step by step:[/INST]
  Sure, I can help you with that! Here's my reasoning step by step:

1. The summary states that Mr. Katter said he would quit after being accused of domestic violence.
2. The article states that the Government believes Mr. Gordon would quit after being recently accused of domestic violence.
3. The key difference between the summary and the article is that the name "Mr. Katter" is mentioned in the summary, but there is no mention of Mr. Katter in the article.
4. Therefore, the summary is not consistent with the article because the article does not me

"\nOriginal:\n1. The summary states that Mr Katter would quit after being accused of domestic violence. \n2. However, the article states that the Government believes Mr Gordon would quit after being recently accused of domestic violence. \n3. The inconsistency is due to the use of the word'recently'. The article implies that the Government believes Mr Gordon would quit after being recently accused, while the summary states that Mr Katter would quit after being accused. \n4. To maintain consistency, the summary should use the same word as the article,'recently'.\n\nCorrected (The major error is not recently)\n1. The summary states that Mr Katter would quit after being accused of domestic violence. \n2. However, the article states that the Government believes Mr Gordon would quit after being recently accused of domestic violence. \n3. The inconsistency is due to the error that the article implies that the Government believes **Mr Gordon** would quit, while the summary states that **Mr Ka

In [9]:
print(llm_generate(template.format(article=doc2, summary=sum2)))

"""
Original:
1. The summary states that Josep Maria Bartomeu says the La Liga leaders are very happy with him.
2. The article states that Bartomeu has insisted that the La Liga leaders have no plans to replace Luis Enrique and they're "very happy" with him.
3. The wording of the summary is different from the wording of the article. In the summary, it is stated that the La Liga leaders are happy with Bartomeu, while in the article, it is stated that the La Liga leaders have no plans to replace Luis Enrique and are happy with him.
4. The reason for the inconsistency is that the summary has misinterpreted the article. The article does not say that the La Liga leaders are happy with Bartomeu, but rather that they have no plans to replace Luis Enrique and are happy with him.
5. Therefore, the summary is not consistent with the article, and the inconsistency is due to a misinterpretation of the article's content.
"""

<s>[INST] Note that consistency means all information in the summary is supported by the article. 
It's known that the following summary is not consistent with the article. 
Find out the why.

Summary: 
Josep Maria Bartomeu says the La Liga leaders are very happy with him. 

Article: 
Barcelona club president Josep Maria Bartomeu has insisted that the La Liga leaders have no plans to replace Luis Enrique and they're `very happy' with him. 

Explain your reasoning step by step:[/INST]

1. The summary states that Josep Maria Bartomeu says the La Liga leaders are very happy with him.
2. The article states that Bartomeu has insisted that the La Liga leaders have no plans to replace Luis Enrique and they're "very happy" with him.
3. The wording of the summary is different from the wording of the article. In the summary, it is stated that the La Liga leaders are happy with Bartomeu, while in the article, it is stated that the La Liga leaders have no plans to replace Luis Enrique and are happ

"\nOriginal:\n1. The summary claims that Bartomeu is happy with himself, which contradicts the article's claim that the La Liga leaders are very happy with him. \n2. The article provides evidence for the claim that the La Liga leaders are very happy with him, while the summary does not. \n3. Therefore, the summary is not consistent with the article.\n\nCorrected (Both texts did not claim Bartomeu is happy with himself):\n1. The summary claims that Bartomeu says the La Liga leaders are very happy with him.\n2. The article provides evidence for the claim that the La Liga leaders have no plans to replace Luis Enrique and they're `very happy' with him. \n3. Therefore, the summary is not consistent with article on that the summary implies the La Liga leaders are very happy with **Bartomeu**, but the article implies the La Liga leaders are very happy with **Luis Enrique**.\n"

In [10]:
print(llm_generate(template.format(article=doc3, summary=sum3)))

"""
Original:
1. The summary states that goldfish are being caught weighing up to 8kg and one meter in length.
2. The article states that goldfish are being caught weighing up to 2kg and koi carp up to 8kg and one meter in length.
3. The article does not mention goldfish weighing up to 8kg and one meter in length, only koi carp.
4. Therefore, the summary is not consistent with the article because it claims that goldfish are being caught weighing up to 8kg and one meter in length, which is not supported by the article.
5. The reason for this inconsistency is likely a mistake or misinterpretation of the information in the article. The article specifically mentions koi carp, not goldfish, as the fish being caught weighing up to 8kg and one meter in length.
"""

<s>[INST] Note that consistency means all information in the summary is supported by the article. 
It's known that the following summary is not consistent with the article. 
Find out the why.

Summary: 
Goldfish are being caught weighing up to 8kg and one metre in length. 

Article: 
Goldfish are being caught weighing up to 2kg and koi carp up to 8kg and one metre in length. 

Explain your reasoning step by step:[/INST]

1. The summary states that goldfish are being caught weighing up to 8kg and one meter in length.
2. The article states that goldfish are being caught weighing up to 2kg and koi carp up to 8kg and one meter in length.
3. The article does not mention goldfish weighing up to 8kg and one meter in length, only koi carp.
4. Therefore, the summary is not consistent with the article because it claims that goldfish are being caught weighing up to 8kg and one meter in length, which is not supported by the article.

The reason for this inconsistency is likely a mistake or misinte

'\nOriginal:\n1. The article states that goldfish can weigh up to 8kg, while the summary states that koi carp can weigh up to 8kg. \n2. The article also states that koi carp can reach a length of 1 metre, while the summary states that goldfish can reach a length of up to 8cm. \n3. Therefore, the summary is not consistent with the article.\n\nCorrected (Hallucination):\n1. The article states that **goldfish** can weigh up to 8kg, while the summary states that **koi carp** can weigh up to 8kg. \n2. Therefore, the summary is not consistent with the article.\n'

In [11]:
print(llm_generate(template.format(article=doc4, summary=sum4)))

"""
Original
1. The summary states that Paul Merson had a dig at Andros Townsend after scoring for England against Italy.
2. The article states that Paul Merson had a dig at Andros Townsend after his appearance for Tottenham against Burnley.
3. These two statements are not consistent, as Andros Townsend did not play for England against Italy.
4. Therefore, the summary is not consistent with the article, and the reason for this inconsistency is that the summary incorrectly states that Andros Townsend played for England against Italy.

Corrected (Hallucination)
1. The summary states that Paul Merson had a dig at Andros Townsend after scoring for England against Italy.
2. The article states that Paul Merson had a dig at Andros Townsend after his appearance for Tottenham against Burnley.
3. These two statements are not consistent, as summary states the event happened after scoring for England against Italy, while the article says after his appearance for Tottenham against Burnley.
4. Therefore, the summary is not consistent with the article, and the reason for this inconsistency is that the summary incorrectly states the time when Paul Merson had a dig at Andros Townsend.
"""

<s>[INST] Note that consistency means all information in the summary is supported by the article. 
It's known that the following summary is not consistent with the article. 
Find out the why.

Summary: 
Paul Merson had another dig at andros townsend after scoring for England against Italy. 

Article: 
Paul Merson had another dig at Andros Townsend after his appearance for Tottenham against Burnley ...Townsend hit back at Merson on Twitter after scoring for England against Italy. 

Explain your reasoning step by step:[/INST]

1. The summary states that Paul Merson had a dig at Andros Townsend after scoring for England against Italy.
2. The article states that Paul Merson had a dig at Andros Townsend after his appearance for Tottenham against Burnley.
3. These two statements are not consistent, as Andros Townsend did not play for England against Italy.
4. Therefore, the summary is not consistent with the article, and the reason for this inconsistency is that the summary incorrectly state

'\nOriginal\n1. The summary states that Paul Merson had another dig at Andros Townsend after scoring for England against Italy.\n2. However, the article mentions that Andros Townsend was not even in the match against Italy.\n3. Therefore, the summary is not consistent with the article.\n\nCorrected (Hallucination)\n1. The summary states that Paul Merson had another dig at Andros Townsend after scoring for England against Italy.\n2. However, the article mentions that Paul Merson had another dig at Andros Townsend after his appearance for Tottenham against Burnley.\n3. Therefore, the summary is not consistent with the article on the time of event where the summary states **after scoring for England against Italy**, while the article states **after his appearance for Tottenham against Burnley**.\n'

In [12]:
print(llm_generate(template.format(article=doc5, summary=sum5)))

"""
Original
1. The summary states that the cargo ship "came to its aid" and that the boat "may have capsized after being touched or swamped."
2. However, the article states that the cargo ship "came to its aid" and that the boat "may have capsized after being touched or swamped" due to the cargo ship's actions.
3. The summary implies that the cargo ship caused the boat to capsize, while the article suggests that the cargo ship's actions may have contributed to the boat's capsizing.
4. The summary is not consistent with the article because it does not accurately convey the information provided in the article. The article does not state that the cargo ship intentionally caused the boat to capsize, but rather that its actions may have contributed to the boat's capsizing.
5. Therefore, the summary is not a consistent representation of the information in the article, and it may be misleading to readers.

Corrected:
1. The summary states hundreds of migrants on board may have capsized.
2. The article mentions the boat with hundreds of migrants on board may have capsized.
3. The summary is not consistent article because the summary says **migrants** may have capsized, while the article states **boat** may have capsized.
"""

<s>[INST] Note that consistency means all information in the summary is supported by the article. 
It's known that the following summary is not consistent with the article. 
Find out the why.

Summary: 
hundreds of migrants on board may have capsized after being touched or swamped by a cargo ship that came to its aid. 

Article: 
Catania, Italy (CNN)The boat that sank in the Mediterranean over the weekend with hundreds of migrants on board may have capsized after being touched or swamped by a cargo ship that came to its aid, a U.N. official said. Carlotta Asami, a spokeswoman for the United Nations' refugee agency, made the comments to CNN early Tuesday after she and two of her colleagues had spoken to multiple survivors from the disaster who arrived in Catania, Italy. 

Explain your reasoning step by step:[/INST]

1. The summary states that the cargo ship "came to its aid" and that the boat "may have capsized after being touched or swamped."
2. However, the article states that the car

'\nOriginal\nUnable to produce reasonning...\n\nCorrected \n1. The summary states hundreds of migrants on board may have capsized.\n2. The article mentions the boat with hundreds of migrants on board may have capsized.\n3. The summary is not consistent article because the summary says **migrants** may have capsized, while the article states **boat** may have capsized.\n'